# Preloading


In [1]:
!pip3 install pytrends
!pip3 install opencc-python-reimplemented
from pytrends.request import TrendReq
import pandas as pd
import jieba
import jieba.analyse
import jieba.posseg as pseg
from opencc import OpenCC
from concurrent.futures import ThreadPoolExecutor
import time
from google.colab import drive
drive.mount('/content/drive/')

f=open('/content/drive/MyDrive/raw_txt.txt','r')
lines=f.readlines()

sym_words=[]
sym_class_words=[]
#同義詞詞集sym_words 相關詞詞集sym_class_words
for line in lines:
    line=line.replace('\n','')
    items=line.split(' ')
    index=items[0]
    if(index[-1]=='='):
        sym_words.append(items[1:])

    if (index[-1] == '#'):
        sym_class_words.append(items[1:])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 5.8 MB/s eta 0:00:00
Mounted at /content/drive/


# Setting

In [2]:
#閥值
if_popular = 0.1
#線程數
how_fast = 10

howlong_sleep = 3

# Define


In [6]:
def get_sym(w,word_set):
    # w:  input word
    # word_set: 同义词词集或相关词词集
    results=[]
    if(len(w)==1):
        for each in word_set:
            for word in each:
                if w == word:
                    results.append(each)
                    break
    else:
        for each in word_set:
            for word in each:
                if w in word:
                    results.append(each)
                    break
    return results

#find good tag

def get_good_friends(friend):
  pytrend = TrendReq(hl='en-US', tz=360, timeout=(10,25))

  global tag_pocket

  cc = OpenCC('s2twp')
  friend = cc.convert(friend)
  if tag_master == friend: return

  keywords = [tag_master, friend]
  pytrend.build_payload(kw_list=keywords,cat=0,timeframe='today 1-m',geo='TW',gprop='')

  df = pd.DataFrame(pytrend.interest_over_time())
  print(tag_master,friend)
  print(df[tag_master].mean(),df[friend].mean())

  if if_popular*df[tag_master].mean() < df[friend].mean():
    tag_pocket.append(friend)
  time.sleep(howlong_sleep)
  print()
  return


# Input



In [ ]:
ps=[]
sen=[]


input2=input('輸入一段文字:')

cc = OpenCC('tw2sp')
input2 = cc.convert(input2)
tag_pocket = []
tag_master = ''

keywords = jieba.analyse.extract_tags(input2, topK=10, withWeight=True, allowPOS=('n','nr','ns','nrt','nrf','nrj'))


for item,v in keywords:
  test_list = []
  precache=get_sym(item, sym_words)

  cc = OpenCC('s2twp')
  tag_master=cc.convert(item)
  tag_pocket.append(tag_master)

  print(tag_master)

  if len(precache):
    print(precache)
    for j in precache:
      for i in j:
        test_list.append(i)
  else:
    continue

  test_list.remove(item)
  print(test_list)

  with ThreadPoolExecutor(max_workers=how_fast) as executor:
    executor.map(get_good_friends, test_list)


#print(ps)

輸入一段文字:80/20法則，起源於20世紀初由義大利統計學家提出。他指出，在任何特定群體中，重要因素通常只占少數，而不重要因素則占多數。因此，只要能控制少數重要因素，就能控制全局。這個原理經過多年演化，成為當今管理學界熟知的二八法則，即80%的公司利潤來自20%的重要客戶，其餘20%的利潤來自80%的普通客戶。
法則
[['规律', '法则', '公例', '公理', '原理', '常理', '公设'], ['经济规律', '经济法则'], ['自然规律', '自然法则']]
['规律', '公例', '公理', '原理', '常理', '公设', '经济规律', '经济法则', '自然规律', '自然法则']
法則 原理
19.59375 80.0
法則 常理
72.28125 1.1875
法則 規律
72.28125 5.3125
法則 公設
72.28125 25.84375
法則 經濟法則
72.28125 0.09375
法則 公例
72.28125 0.0625






因素
[['因素', '要素', '元素', '素']]
['要素', '元素', '素']
客戶
利潤
[['利润', '赢利', '盈利', '利', '赚头', '创收', '净收入', '净利润', '纯利润', '实利']]
['赢利', '盈利', '利', '赚头', '创收', '净收入', '净利润', '纯利润', '实利']


# Output


In [5]:
#Output
meow=list(set(tag_pocket))
meow2=[]
cc = OpenCC('tw2sp')


for i in meow:
  for w,tag in jieba.posseg.cut(cc.convert(i)):
    if len(w) == 1:
      if not 'n' in tag:
        continue
    meow2.append(OpenCC('s2twp').convert(w))

print(meow2)
sen=''
print('for IG')
print()
for i in meow2:
  sen=sen+'#'+str(i)
print(sen)
print()
sen=''
print('for YT')
print()
for i in meow2:
  sen=sen+str(i)+','
print(sen)

['法則', '混亂', '自然法則', '風', '風俗', '心態', '證明', '民俗', '常理', '驗證', '錯雜', '習慣', '傳統', '狼藉', '雜亂', '冗雜', '淆亂', '公理', '認證', '神經遞質', '公設', '習俗', '證實', '情緒', '程度', '經濟', '規律', '亂七八糟', '心情', '說明', '錯亂', '混雜', '水準', '證', '原理', '糊塗', '作證', '規律', '水平', '紊亂', '印證', '零亂']
for IG

#法則#混亂#自然法則#風#風俗#心態#證明#民俗#常理#驗證#錯雜#習慣#傳統#狼藉#雜亂#冗雜#淆亂#公理#認證#神經遞質#公設#習俗#證實#情緒#程度#經濟#規律#亂七八糟#心情#說明#錯亂#混雜#水準#證#原理#糊塗#作證#規律#水平#紊亂#印證#零亂

for YT

法則,混亂,自然法則,風,風俗,心態,證明,民俗,常理,驗證,錯雜,習慣,傳統,狼藉,雜亂,冗雜,淆亂,公理,認證,神經遞質,公設,習俗,證實,情緒,程度,經濟,規律,亂七八糟,心情,說明,錯亂,混雜,水準,證,原理,糊塗,作證,規律,水平,紊亂,印證,零亂,
